# 1. Data cleaning

## Problem statement

This project is meant as an exercise in basic classification as a training after reading the first three chapters of Hands-On Machine Learning with Scikit-Learn, Keras and Tensorflow by Géron.

### Learning goal: practise implementing a simple end-to-end classification task
Rather than try and achieve very performant results (e.g. optimising an accuracy metric) a focus is to cover all major steps of a typical classification task:
1. Data collection
2. Data cleaning and preprocessing
3. Exploratory data analysis
4. Training of Machine Learning algorithms
5. Optimising the chosen ML technique
6. Drawing insights and conclusions
7. Deployment

The sequential steps are described in separate notebooks.

### Business goal: detect customers that are likely to cancel their credit-card subscription
I found the dataset on Kaggle:
https://www.kaggle.com/sakshigoyal7/credit-card-customers

It comes from a real-life scenario: a bank manager is unhappy that quite a few customers have been cancelling their credit card membership. He/She wants to find out if it's possible to detect such discontent customers early on and win them over by proposing attractive offers before it's too late.
The data has been collected by a Kaggle user, however, it requires some preparation due to:
- text rather than numeric values
- missing values

## Load utilities and programming set up

Let's start by loading the useful libraries, specifying the input data file, and preparing a utility function that will be used to save any figures later on.

In [42]:
# Usual imports
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
assert sklearn.__version__ >= "0.20"

# Pretty plotting
%matplotlib inline
import matplotlib as mpl
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Data file name (easy to change for a similar project)
DATA_FILE = "BankChurners.csv"

# Where to save the figures
PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
os.makedirs(IMAGES_PATH, exist_ok=True)  # It's fine if the directory already exists (No action needed)

def save_fig(fig_name, tight_layout=True, fig_extension="png", resolution=300):
    "Save figure ensurring correct directory, extension, resolution and layout"
    path = os.path.join(IMAGES_PATH, fig_name + "." + fig_extension)
    print("Saving figure", fig_name)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

## Load the data

Below I define and call a utility function that loads the data into a pandas DataFrame.
Then, we can have a glimpse at the data.

In [43]:
def load_data(data_path = PROJECT_ROOT_DIR, data_file = DATA_FILE):
    "Load csv data into a pandas DataFrame"
    csv_path = os.path.join(data_path, data_file)
    return pd.read_csv(csv_path)
df = load_data()
df.head(100)

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0.000093,0.999910
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0.000057,0.999940
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0.000021,0.999980
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0.000134,0.999870
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0.000022,0.999980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,719712633,Existing Customer,64,M,1,Graduate,Married,Less than $40K,Blue,52,...,1709.0,895,814.0,1.656,1673,32,0.882,0.524,0.000343,0.999660
96,772629333,Existing Customer,45,M,3,Graduate,Married,$40K - $60K,Blue,35,...,3454.0,1200,2254.0,0.597,1313,30,0.304,0.347,0.000226,0.999770
97,720336708,Existing Customer,53,M,3,Doctorate,Married,$40K - $60K,Blue,35,...,3789.0,1706,2083.0,1.047,1609,42,0.680,0.450,0.000270,0.999730
98,802013583,Existing Customer,56,M,3,College,Married,$120K +,Blue,50,...,9689.0,2250,7439.0,0.576,1158,19,0.727,0.232,0.000042,0.999960


In [44]:
# Show percentage of attrited and existing customers
df.groupby("Attrition_Flag").count() / df.shape[0] * 100

,CLIENTNUM,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
Attrition_Flag,,,,,,,,,,,,,,,,,,,,,
Attrited Customer,16.065962,16.065962,16.065962,16.065962,16.065962,16.065962,16.065962,16.065962,16.065962,16.065962,...,16.065962,16.065962,16.065962,16.065962,16.065962,16.065962,16.065962,16.065962,16.065962,16.065962
Existing Customer,83.934038,83.934038,83.934038,83.934038,83.934038,83.934038,83.934038,83.934038,83.934038,83.934038,...,83.934038,83.934038,83.934038,83.934038,83.934038,83.934038,83.934038,83.934038,83.934038,83.934038


We notice that:
- The first column is a unique client number; we won't use it
- The second one is the client category: either "Existing Customer" or "Attrited Customer". We'll use it as a label for the classification algorithms. Note the imbalance between the existing (84%) and attrited (16%) customers. Hence the problem we're dealing with is actually **imbalanced classification** (more on it below)
- The next columns are various characteristics of each customer, potentially feautures of an ML algorithm
- Some columns are text-based. They need to be translated into numeric values to be ready for an ML technique.

### Imbalanced classification
Having the class ratio significantly different from 1:1 (as it is the case here) may cause an ML algorithm to ignore the minority class. This is obviously undesirable.
While there's no magic solution to the problem, there exist a number of strategies to try out:
1. Use a more suitable performance metric than accuracy, e.g. confusion matrix
2. Over(under) sample the minority (majority) class.
3. Syntehise the feautures of the minory class to manufacture plausible new instances of the minority class (the SMOTE technique)
3. Choose an algorithm that can mitigate the imbalanced class representation
4. Gather more data (not feasible if the imbalance is intrinsic to the process as it is the case in churn/anomaly/fraud detection, etc.)

More information:
- https://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/
- SMOTE: https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/

For the step-by-step development I'd proceed as follows:
1. Ignore the "elephant in the room" to firstly focus on the overall workflow
2. Use the confusion matrix (or similar) as a peformance measure
3. Pick an easy solution to tackle the imbalanced classification (e.g. over-sample the minority class)

Moreover, the dataset author tells that he's been working on Naive Bayes classifier with no satisfactory outcome.
Therefore the corresponding columns are rather *outputs* than *inputs* (features) and let's drop them together with the client number.

In [45]:
for col in [
    "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1",
    "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2",
    "CLIENTNUM",
    ]:
    print(col)
    df = df.drop(col, axis=1)

Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1
Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
CLIENTNUM


Let's display more information about the dataset with `df.info()`.

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Attrition_Flag            10127 non-null  object 
 1   Customer_Age              10127 non-null  int64  
 2   Gender                    10127 non-null  object 
 3   Dependent_count           10127 non-null  int64  
 4   Education_Level           10127 non-null  object 
 5   Marital_Status            10127 non-null  object 
 6   Income_Category           10127 non-null  object 
 7   Card_Category             10127 non-null  object 
 8   Months_on_book            10127 non-null  int64  
 9   Total_Relationship_Count  10127 non-null  int64  
 10  Months_Inactive_12_mon    10127 non-null  int64  
 11  Contacts_Count_12_mon     10127 non-null  int64  
 12  Credit_Limit              10127 non-null  float64
 13  Total_Revolving_Bal       10127 non-null  int64  
 14  Avg_Op

Peeking at the dataset reveals that there are:
- 20 columns: the label "Attrition_Flag" and 19 features
- 5 columns have non-numerical (i.e. text-based) data (`object` type)
- There are `Unknown` values, which obviously are actual null values and should be dealt with

Now, let's have a look at correlations between feautures.
At this stage some values are non-numerical and, consequently, ignored in correlation computations.
TODO: It would be interesting to digitise the text. Especially correlations with the target labels might prove useful.

In [47]:
corr_matrix = df.corr()
corr_matrix
# unfortunately, no correlations with the label that is still text-based (not numerical)

,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
Customer_Age,1.000000,-0.122254,0.788912,-0.010931,0.054361,-0.018452,0.002476,0.014780,0.001151,-0.062042,-0.046446,-0.067097,-0.012143,0.007114
Dependent_count,-0.122254,1.000000,-0.103062,-0.039076,-0.010768,-0.040505,0.068065,-0.002688,0.068291,-0.035439,0.025046,0.049912,0.011087,-0.037135
Months_on_book,0.788912,-0.103062,1.000000,-0.009203,0.074164,-0.010774,0.007507,0.008623,0.006732,-0.048959,-0.038591,-0.049819,-0.014072,-0.007541
Total_Relationship_Count,-0.010931,-0.039076,-0.009203,1.000000,-0.003675,0.055203,-0.071386,0.013726,-0.072601,0.050119,-0.347229,-0.241891,0.040831,0.067663
Months_Inactive_12_mon,0.054361,-0.010768,0.074164,-0.003675,1.000000,0.029493,-0.020394,-0.042210,-0.016605,-0.032247,-0.036982,-0.042787,-0.038989,-0.007503
Contacts_Count_12_mon,-0.018452,-0.040505,-0.010774,0.055203,0.029493,1.000000,0.020817,-0.053913,0.025646,-0.024445,-0.112774,-0.152213,-0.094997,-0.055471
Credit_Limit,0.002476,0.068065,0.007507,-0.071386,-0.020394,0.020817,1.000000,0.042493,0.995981,0.012813,0.171730,0.075927,-0.002020,-0.482965
Total_Revolving_Bal,0.014780,-0.002688,0.008623,0.013726,-0.042210,-0.053913,0.042493,1.000000,-0.047167,0.058174,0.064370,0.056060,0.089861,0.624022
Avg_Open_To_Buy,0.001151,0.068291,0.006732,-0.072601,-0.016605,0.025646,0.995981,-0.047167,1.000000,0.007595,0.165923,0.070885,-0.010076,-0.538808
Total_Amt_Chng_Q4_Q1,-0.062042,-0.035439,-0.048959,0.050119,-0.032247,-0.024445,0.012813,0.058174,0.007595,1.000000,0.039678,0.005469,0.384189,0.035235


'Credit_Limit' is 99.6% correlated with 'Avg_Open_To_Buy'. Therefore the latter feature can be dropped.

In [48]:
df = df.drop("Avg_Open_To_Buy", axis=1)

## Create train and test sets
Despite we're only at the cleaning stage, arguably it's a good practice to already seperate out the test set from the training one to avoid (analyst's) biases.

Let's use 20% of the dataset as the test sample.
To ensure representative sampling we're using the stratified shuffle split: the feautures distributions should agree between the train and test samples.
However, stratified sampling does not address the above-mentioned _imbalance_ between the class labels.
TODO: Let's keep that in mind and tackle at a later stage.

In [49]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(df, df['Attrition_Flag']):
    strat_train_set = df.loc[train_index]
    strat_test_set = df.loc[test_index]
    
print(strat_train_set['Attrition_Flag'].value_counts() / len(strat_train_set))
print(strat_test_set['Attrition_Flag'].value_counts() / len(strat_test_set))

Existing Customer    0.839279
Attrited Customer    0.160721
Name: Attrition_Flag, dtype: float64
Existing Customer    0.839585
Attrited Customer    0.160415
Name: Attrition_Flag, dtype: float64


### Saving the dataset for the next step
For the sake of clarity, let's save the current data set using the `pickle` module.
In the next notebook we'll further explore the data.

In [50]:
strat_train_set.to_pickle("dataset_train_step1.pickle")
strat_test_set.to_pickle("dataset_test_step1.pickle")